In [1]:
# Import necessary libraries
import xarray as xr
import matplotlib.pyplot as plt
import os
import glob

In [2]:
import sys
# Get the current directory of the notebook (replace this with your notebook's directory if not running interactively)
notebook_dir = os.getcwd()

# Construct the path to the module
module_path = os.path.abspath(os.path.join(notebook_dir, "../src/"))

# Add the module's directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import your module and function
from crop_data import crop_and_plot_data

In [3]:
# Base directory for your datasets
base_dir = "/mnt/d/tom_data/SWOT"

In [4]:
# List files and directories in your data folder
for name in os.listdir(base_dir):
    print(name)


L2_LR_expert
.ipynb_checkpoints
download_SWOT.py
error.log
icebergs
L2_LR_basic
L2_LR_basic.zip
L2_LR_unsmoothed
L2_LR_windwave
L3_LR_expert
output.log
plots


In [5]:
subdir_name = "L2_LR_windwave"
file_pattern="*.nc"

In [6]:
path_pattern = os.path.join(base_dir, subdir_name, file_pattern)

In [7]:
# Define the bounding box for Martha's Vineyard with the given bounds
# center point of Matha's Vinyard: 41°24′N 70°37′W

# lat_min, lat_max = 39.5, 43.5 # Latitude bounds
# lon_min, lon_max = -71.5, -69.5 # Longitude bounds

## Bigger bounding box 

lat_min, lat_max = 38.5, 44. # Latitude bounds
lon_min, lon_max = -73.5, -69.5 # Longitude bounds

lolabox = [lon_min, lon_max, lat_min, lat_max]

In [8]:
file_path = glob.glob(path_pattern)

In [9]:
file_path[:4]

['/mnt/d/tom_data/SWOT/L2_LR_windwave/SWOT_L2_LR_SSH_WindWave_484_005_20230407T235536_20230408T004352_PIB0_01.nc',
 '/mnt/d/tom_data/SWOT/L2_LR_windwave/SWOT_L2_LR_SSH_WindWave_484_006_20230408T004351_20230408T013433_PIB0_01.nc',
 '/mnt/d/tom_data/SWOT/L2_LR_windwave/SWOT_L2_LR_SSH_WindWave_484_007_20230408T013659_20230408T022521_PIB0_01.nc',
 '/mnt/d/tom_data/SWOT/L2_LR_windwave/SWOT_L2_LR_SSH_WindWave_484_008_20230408T022702_20230408T031709_PIB0_01.nc']

In [10]:
img_dir = f"/home/yugao/SWOT_L2/img/{subdir_name}"
processed_dir = f"/home/yugao/SWOT_L2/data/processed/{subdir_name}"

## Initial exploration

In [11]:
def crop_and_plot_wind_data(file_path, lolabox, img_dir, processed_dir):
    """
    Process and plot wind data within a specified bounding box and save the results.

    Parameters:
    - file_path: Path to the NetCDF file.
    - lolabox: Bounding box as a tuple (lon_min, lon_max, lat_min, lat_max).
    - img_dir: Directory to save images.
    - processed_dir: Directory to save processed NetCDF files.
    """
    # Load dataset
    for file in file_path:
        ds = xr.open_dataset(file)
        
        # Adjust longitude coordinates
        ds_expert = ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180))
        
        # Subset the dataset based on the bounding box
        lolasubset = (
            (ds_expert.longitude >= lolabox[0]) &
            (ds_expert.longitude <= lolabox[1]) & 
            (ds_expert.latitude >= lolabox[2]) &
            (ds_expert.latitude <= lolabox[3])
        )
        
        if lolasubset.any():
            ds_expert_sub = ds_expert.where(lolasubset, drop=True)
            
            # Check if required variables exist
            required_vars = ['wind_speed_karin', 'wind_speed_model_u', 'wind_speed_model_v']
            if all(var in ds_expert_sub for var in required_vars):
                # Plotting
                fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': ccrs.PlateCarree()})
                
                # Plot wind speed as background
                wind_speed = ds_expert_sub['wind_speed_karin']
                wind_speed.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='viridis')
    
                # Overlay wind vectors
                stride = 10  # Adjust this based on your data resolution
                q = ax.quiver(
                    ds_expert_sub.longitude[::stride],
                    ds_expert_sub.latitude[::stride],
                    ds_expert_sub['wind_speed_model_u'][::stride],
                    ds_expert_sub['wind_speed_model_v'][::stride],
                    transform=ccrs.PlateCarree(), scale=500,
                    color='white'
                )
    
                ax.coastlines()
                ax.set_title('Wind Speed and Direction')
    
                # Ensure output directories exist
                os.makedirs(img_dir, exist_ok=True)
                os.makedirs(processed_dir, exist_ok=True)
    
                # Save plot
                time_coverage_begin = ds_expert_sub.attrs.get('time_coverage_begin', 'unknown_time')
                safe_time_coverage = time_coverage_begin.replace(':', '').replace('T', '_').replace('Z', '')
                img_file_path = os.path.join(img_dir, f"Wind_{safe_time_coverage}.png")
                plt.savefig(img_file_path, dpi=300, bbox_inches='tight')
                plt.close(fig)
                
                # Save subset as NetCDF
                netcdf_file_path = os.path.join(processed_dir, f"Wind_Data_{safe_time_coverage}.nc")
                ds_expert_sub.to_netcdf(path=netcdf_file_path)
                
                print(f"Processed data and plot saved. \nImage: {img_file_path} \nNetCDF: {netcdf_file_path}")
                
                return ds_expert_sub
                
            else:
                missing_vars = [var for var in required_vars if var not in ds_expert_sub]
                print(f"Missing required variables in dataset: {missing_vars}")
        else:
            print("No data within the specified bounding box.")

    


In [12]:
ds_sub = crop_and_plot_wind_data(file, lolabox, img_dir, processed_dir)

NameError: name 'file' is not defined

In [ ]:
file

In [ ]:
ds = ds_21day

# Extract variables
wind_speed = ds['wind_speed_karin']
u_component = ds['wind_speed_model_u']
v_component = ds['wind_speed_model_v']
latitude = ds['latitude']
longitude = ds['longitude']

# Since plotting every vector can make the plot too dense,
# we'll subsample the data. Adjust the stride as necessary to suit your dataset's resolution and size.
stride = 10  # This means plot every 10th data point

# Create a PlateCarree projection plot
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': ccrs.PlateCarree()})

# Plot wind speed as background
wind_speed.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='viridis')

# Overlay wind vectors
# Ensure longitude, latitude, and the components are correctly aligned; np.meshgrid might be needed depending on your data structure
q = ax.quiver(longitude[::stride], latitude[::stride], u_component[::stride], v_component[::stride], 
              transform=ccrs.PlateCarree(), scale=500, headwidth=3, headlength=5, headaxislength=4.5)

# Adding coastlines for geographic context
ax.coastlines()

plt.title('Wind Speed and Direction')
plt.show()
